In [2]:
import urllib
import requests
import pandas
import json
import os
import sys
import uuid
import random
import pprint as pp
import dotenv as env
env.load_dotenv('../../keys.env')

#<script async src="https://cse.google.com/cse.js?cx=314b1cf0785af4af3">
#</script>
#<div class="gcse-search"></div>

#API Doc here: https://developers.google.com/custom-search/v1/reference/rest/v1/cse/list

True

In [3]:
class google_Search:
    __SEARCH_API_KEY = os.environ.get('SEARCH_API_KEY')
    __cx_id = os.environ.get('SEARCH_CX_ID')
    
    def __init__(self):
        return

    def __sanitize_query(self, query):
        query = query.strip()
        sanitized_query = urllib.parse.quote_plus(query)
        return sanitized_query
    
    def search(self, query, terms='None', num_pages=1, silent_mode=True) -> pandas.DataFrame:
        if(silent_mode):
            original_stdout = sys.stdout   # save the original stdout
            sys.stdout = open(os.devnull, 'w')
        uuids = []
        headlines = []
        links = []
        link_id = []
        max_page = 10   #max pages to which we can run custom search
        if(query!=''):
            query = self.__sanitize_query(query)
            cur_page = 0
            while cur_page < num_pages and cur_page < max_page :
                url = f'https://www.googleapis.com/customsearch/v1?key={google_Search.__SEARCH_API_KEY}&cx={google_Search.__cx_id}&q={query}&exact={terms}&num=10&start={cur_page*10+1}'
                print(f'Running search for {query} on page: {cur_page+1}')
                response = requests.get(url)
                if(response.status_code == 200):
                    data = response.json()
                    print(f'Found {len(data['items'])} on page {cur_page+1}')
                    for item in data['items']:
                        uuids.append(uuid.UUID(int=random.getrandbits(128), version=4))
                        headlines.append(item['pagemap']['metatags'][0]['og:description']) #index called 'og:description  has a name of the link'
                        links.append(item['link'])
                        link_id.append(item['link'][-19:][:14])
                else:
                    print(f'Error: {response.status_code} at run page: {cur_page+1}')
                    break
                cur_page += 1 
            print('Combining all outcomes...')
            search_headlines = pandas.DataFrame({
                                    'uuid': uuids, 
                                    'title': headlines,     #Don't change these column names, they are important for merging later 
                                    'link': links,
                                    'link_id': link_id
                                    })
            search_headlines.insert(1, 'site', 'Livemint')  #Because currently our search engine is set to search on LiveMint.
            
            print('Deduping links...')
            search_headlines.drop_duplicates(subset=['link_id'], keep='first', inplace=True, ignore_index=True)
            if(silent_mode):
                sys.stdout.close()
                sys.stdout = original_stdout    #restore the original stdout, other wise it will fail other prints.
            return search_headlines
        else:
            if(silent_mode):
                sys.stdout.close()
                sys.stdout = original_stdout
            return None

In [4]:
#one line tester

#search_engine = google_Search()
#links = search_engine.search(query='Vaishali Parekh', terms='Buy', num_pages=1, silent_mode=True)
#pp.pprint(links)

                                   uuid      site  \
0  dafc104a-5e23-4768-a64d-44df9951b250  Livemint   
1  dd81bffb-3f66-4f9c-91e2-b2b14883cac8  Livemint   
2  8e3168a6-a4b8-4886-88d6-4f5323a01355  Livemint   
3  77c12b20-88a2-4c00-9eed-16955c21d91b  Livemint   
4  59adad60-de95-4440-8ac6-2f87fb9f806a  Livemint   
5  150dd104-33d2-4fa7-b341-ae01e4b0198a  Livemint   
6  a1e59a12-c453-4805-bc4a-59347753b5eb  Livemint   
7  4ad0f1e8-c584-4968-bd00-54d61b080648  Livemint   
8  41f9d3fa-7a94-4a3f-9864-8c6fab4dd30b  Livemint   
9  c345849c-cf11-4c24-b780-0875e357736c  Livemint   

                                               title  \
0  Buy or sell: Vaishali Parekh recommends three ...   
1  Buy or sell: Vaishali Parekh recommends three ...   
2  Buy or sell: Baishali Parekh recommends three ...   
3  Stocks to buy under  ₹100: Vaishali Parekh rec...   
4  Buy or sell: Amid escalating tension of India-...   
5  Buy or sell stocks: Vaishali Parekh recommends...   
6  Buy or sell stocks: V